<a href="https://colab.research.google.com/github/jarvisx17/EPO/blob/main/EPO_model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Step 1: Preprocess the data
df = pd.read_csv('/content/Final_df2_EPO.csv')  # Replace 'your_dataset.csv' with your actual dataset file
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128  # Maximum sequence length for input text

texts = df['Cited Text'].tolist()
claims = df['rel_Claim'].tolist()
labels = df['Label'].tolist()

In [ ]:
#Label Encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1])

In [ ]:
input_ids = []
attention_masks = []

for text, claim in zip(texts, claims):
    encoded_dict = tokenizer.encode_plus(
        text,
        claim,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Step 2: Split the data
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Step 3: Load a pre-trained transformer model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 4: Create a classification model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Step 5: Train the model
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{epochs} - Average Loss: {avg_loss}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30 - Average Loss: 0.5965866521000862
Epoch 2/30 - Average Loss: 0.49030841141939163
Epoch 3/30 - Average Loss: 0.5268676578998566
Epoch 4/30 - Average Loss: 0.4066370651125908
Epoch 5/30 - Average Loss: 0.34893446415662766
Epoch 6/30 - Average Loss: 0.26952309533953667
Epoch 7/30 - Average Loss: 0.2747241370379925
Epoch 8/30 - Average Loss: 0.21762023866176605
Epoch 9/30 - Average Loss: 0.14545208029448986
Epoch 10/30 - Average Loss: 0.1153098652139306
Epoch 11/30 - Average Loss: 0.10462047532200813
Epoch 12/30 - Average Loss: 0.07863035891205072
Epoch 13/30 - Average Loss: 0.04188944539055228
Epoch 14/30 - Average Loss: 0.03593367012217641
Epoch 15/30 - Average Loss: 0.03396087745204568
Epoch 16/30 - Average Loss: 0.023504230193793774
Epoch 17/30 - Average Loss: 0.019332364667207003
Epoch 18/30 - Average Loss: 0.01565191289409995
Epoch 19/30 - Average Loss: 0.0119567452929914
Epoch 20/30 - Average Loss: 0.009642744902521372
Epoch 21/30 - Average Loss: 0.008565668947994709
Epo

In [ ]:
# Step 6: Evaluate the model
model.eval()
total_predictions, correct_predictions = 0, 0

for batch in train_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_masks, labels = batch

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

    predicted_labels = torch.argmax(logits, dim=1)
    total_predictions += labels.size(0)
    correct_predictions += (predicted_labels == labels).sum().item()

accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy}')

Accuracy: 0.6545454545454545


In [ ]:
# Step 6: Evaluate the model
model.eval()
total_predictions, correct_predictions = 0, 0

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_masks, labels = batch

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

    predicted_labels = torch.argmax(logits, dim=1)
    total_predictions += labels.size(0)
    correct_predictions += (predicted_labels == labels).sum().item()

accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy}')

Accuracy: 0.5


In [ ]:
# torch.save(model, 'model_weights.pth')
# Save the model weights
torch.save(model.state_dict(), 'model_weights.pth')

# Save the entire model (including architecture and optimizer state)
torch.save(model, 'model.pth')

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load the trained model weights
model.load_state_dict(torch.load('model_weights.pth'))  # Replace 'model_weights.pth' with the path to your trained model weights

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Text and claim to be classified
text = df.iloc[0][0]
claim = df.iloc[0][1]
actual = df.iloc[0][2]

# Tokenize and encode the input
input_ids = tokenizer.encode(text, claim, add_special_tokens=True, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
input_ids = input_ids.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

predicted_label = predicted_labels.item()
if predicted_label == 0:
    prediction = "Negative"
else:
    prediction = "Positive"

print(f"Prediction: {prediction}  \nActual : {actual}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Prediction: Negative  
Actual : Negative
